In [1]:
import tensorflow.keras as keras
import numpy as np
import importlib

import kaggle_functions as kaggle
import model10_wideresnet
importlib.reload(kaggle); 
importlib.reload(model10_wideresnet);
from model10_wideresnet import Model

model_number = 'model10'

### Create datasets

In [2]:
train_dataset, valid_dataset, valid_labels = kaggle.load_train_as_dataset()

batch_size = 32
train_dataset_augmented, epoch_length = kaggle.augment_dataset(train_dataset, batch_size)

In [ ]:
kaggle.show_images(train_dataset_augmented, 9)

### Train new model

In [ ]:
Model().build(None).summary()

In [15]:
# Fit model
# See {model}.py for specific training instructions, like epochs or valid_patience
model_name = 'WideResNet_6'

model, history = kaggle.train_model(
    Model().build(None), train_dataset_augmented, valid_dataset, 
    epochs=120, valid_patience=16, epoch_length=epoch_length)

Epoch 1/200
298/298 [==============================] - 240s 788ms/step - loss: 4.8863 - accuracy: 0.2031 - val_loss: 4.9331 - val_accuracy: 0.1644 - lr: 5.0000e-04
Epoch 2/200
298/298 [==============================] - 234s 786ms/step - loss: 4.0953 - accuracy: 0.2599 - val_loss: 5.7734 - val_accuracy: 0.1102 - lr: 5.0000e-04
Epoch 3/200
298/298 [==============================] - 225s 756ms/step - loss: 3.4746 - accuracy: 0.2778 - val_loss: 4.1434 - val_accuracy: 0.2931 - lr: 5.0000e-04
Epoch 4/200
298/298 [==============================] - 212s 711ms/step - loss: 3.0149 - accuracy: 0.3025 - val_loss: 4.3981 - val_accuracy: 0.1800 - lr: 5.0000e-04
Epoch 5/200
298/298 [==============================] - 212s 711ms/step - loss: 2.7462 - accuracy: 0.3082 - val_loss: 5.0687 - val_accuracy: 0.2140 - lr: 5.0000e-04
Epoch 6/200
298/298 [==============================] - 212s 711ms/step - loss: 2.5631 - accuracy: 0.3261 - val_loss: 6.4968 - val_accuracy: 0.2132 - lr: 5.0000e-04
Epoch 7/200
298/

In [16]:
# Save model
model.save(f'models/{model_number}/{model_name}')

# Plot model statistics during training
kaggle.plot_model_history(history, [['accuracy', 'val_accuracy'], ['loss', 'val_loss']])

INFO:tensorflow:Assets written to: models/model10/WideResNet_6\assets


C:\Users\Guillaume\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\Guillaume\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [3]:
model = keras.models.load_model(f'models/{model_number}/WideResNet_2_73')

### Fine-tune

In [ ]:
# Fine-tune model
fine_model, history = kaggle.fine_tune_model(
    model,
    train_dataset.batch(32), valid_dataset, 
    epochs=2, epoch_length=epoch_length, learning_rate=1E-5)

In [ ]:
# Save fine-tuned model
fine_model_name = 'WideResNet_5_84'
fine_model.save(f'models/{model_number}/{fine_model_name}')

### Evaluate

In [ ]:
model_name = ''
model = keras.models.load_model(f'models/{model_number}/{model_name}')
test_pred = np.argmax(model.predict(valid_dataset.batch(128)), axis=1)

kaggle.print_accuracy(valid_labels, test_pred)
kaggle.plot_confusion_matrix(valid_labels, test_pred)

### Generate test predictions

In [ ]:
model_name = 'WideResNet_3'
kaggle.generate_test_pred_filepath(f'models/{model_number}/{model_name}')

In [11]:
model_name = 'Stacking_0'
kaggle.generate_test_pred(model, f'models/{model_number}/{model_name}_test_pred.csv')

### Hypertune models

In [ ]:
tuner = kaggle.hypertune_model(
    Model(), train_dataset.batch(32).cache(), valid_dataset, 
    model_number, 'hypertuner2021-12-03', trials=4, 
    epochs=11, valid_patience=3, epoch_length=epoch_length)

In [4]:
tuner = kaggle.load_hypertuner(Model(), model_number, 'hypertuner2021-12-03')

INFO:tensorflow:Reloading Oracle from existing project models/model10\hypertuner2021-12-03\oracle.json
INFO:tensorflow:Reloading Tuner from models/model10\hypertuner2021-12-03\tuner0.json
Results summary
Results in models/model10\hypertuner2021-12-03
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_dropout: 0.5
k: 9
Score: 0.8048780560493469
Trial summary
Hyperparameters:
conv_dropout: 0.3
k: 9
Score: 0.8015138506889343
Trial summary
Hyperparameters:
conv_dropout: 0.5
k: 6
Score: 0.7211942672729492


In [10]:
model = tuner.get_best_models(1)[0]